In [1]:
import tensorflow
import numpy as np
import keras
import os
from utility_functions import load_dataset

path_to_file = '/home/alexandr/data/audio/'
path_to_file_test = '/home/alexandr/data/test/'
path_to_file_unknown = '/home/alexandr/data/unknown/'

Using TensorFlow backend.


In [2]:
# extract classes from txt file

raw_classes = np.loadtxt('/home/alexandr/data/meta/meta.txt', dtype='str')
#print(raw_classes)
classes = np.delete(raw_classes, [1, 2, 3], 1)
text_labels = ['background', 'bags', 'door', 'keyboard', 'knocking_door', 'ring', 'speech', 'tool']

for index, label in enumerate(text_labels):
    classes[classes == label] = index

dict_classes = dict(classes)
classes_test = np.loadtxt('./meta-test.txt',dtype='str')
dict_classes_test = dict(classes_test)

raw_data_train = np.array(load_dataset(path_to_file, dict_classes))
raw_data_test = np.array(load_dataset(path_to_file_test, dict_classes_test))
raw_data_unknown = np.array(load_dataset(path_to_file_unknown))


In [3]:
from detector import calculate_RMS
overall_RMS = 0
list_RMS = []
background = raw_data_train[raw_data_train[:,1] == '0']
for frame in background:
    overall_RMS += calculate_RMS(frame[0].astype(int))
    list_RMS.append(calculate_RMS(frame[0].astype(int)))
mean_background = overall_RMS/len(background)
print(mean_background)

events1 = raw_data_train[(raw_data_train[:,1] == '1')]
events2 = raw_data_train[(raw_data_train[:,1] == '2')]
events3 = raw_data_train[(raw_data_train[:,1] == '3')]
events4 = raw_data_train[(raw_data_train[:,1] == '4')]
events5 = raw_data_train[(raw_data_train[:,1] == '5')]
events6 = raw_data_train[(raw_data_train[:,1] == '6')]
events7 = raw_data_train[(raw_data_train[:,1] == '7')]


events = np.concatenate([events1,events2, events3, events4, events5, events6, events7])

# for frame in events:
#     list_RMS.append(calculate_RMS(frame[0].astype(int)))
min_events = np.min(list_RMS)

print(events.shape)

KeyboardInterrupt: 

In [20]:
print(np.sort(list_RMS))

[  1.36164045   1.3724955    1.37888762 ... 488.40743365 493.50026942
 501.93080995]


In [3]:
# 
# cropped_data_train = []
# for sound in raw_data_train[:,0]:
#     start_point = detect_startpoint(sound)
#     cropped_data_train.append(sound[start_point:])
# cropped_data_train = np.array(cropped_data_train)
# print (1)
# cropped_data_test = []
# for sound in raw_data_test[:,0]:
#     start_point = detect_startpoint(sound)
#     cropped_data_test.append(sound[start_point:])
# cropped_data_test = np.array(cropped_data_test)
cropped_data_train =raw_data_train
cropped_data_test = raw_data_test
cropped_data_unknown = raw_data_unknown

In [4]:
# first second segmentation
from detector import detect_event
from utility_functions import find_longest, padding_data, get_random_segment

# max_length = find_longest(raw_data)
# pad_data = padding_data(raw_data, max_length)
segmented_data_train = []

# TODO remove index
for index, sound in enumerate(cropped_data_train[:,0]):
    fragment = get_random_segment(sound)
    segmented_data_train.append(fragment)
    
    RMS, isEvent = detect_event(fragment)
    if isEvent:
        print(detect_event(fragment))
segmented_data_train = np.array(segmented_data_train)


segmented_data_test= []

for index, sound in enumerate(cropped_data_test[:,0]):
    fragment = get_random_segment(sound)
    segmented_data_test.append(fragment)
    detect_event(fragment)
segmented_data_test = np.array(segmented_data_test)

segmented_data_unknown = []

for index, sound in enumerate(cropped_data_unknown):
    segmented_data_unknown.append(get_random_segment(sound))

(15192.722284164529, True)


(19065.559526279332, True)


(345039.96372671594, True)


(98565.7230082022, True)
(59569.990155356994, True)
(389177.8432480689, True)


(359036.3120664211, True)
(305444.2515922127, True)


...

(445831.6719439382, True)
(34663.40277482209, True)
(64055.90700755631, True)


(334484.1928056009, True)


(17642.07916151136, True)
(21464.3741992114, True)
(74314.75060117047, True)


(169442.5091042096, True)


KeyboardInterrupt: 

In [5]:
# clean memory
pad_data = None
raw_data = None
extracted_features = None

In [7]:
print(np.array(segmented_data_train).shape)
print(np.array(segmented_data_test).shape)
print(np.array(raw_data_train).shape)
print(np.array(raw_data_test).shape)
print(np.array(segmented_data_unknown).shape)


(11307, 48000)
(473, 48000)
(11307, 2)
(473, 2)
(137, 48000)


In [6]:
import librosa
from speechpy.feature import lmfe

extracted_features_train = []
extracted_features_test = []
extracted_features_unknown = []

def extract_feature(mode='mfcc'):
    if mode == 'mfcc':
        for sound in segmented_data_train:
            feature = librosa.feature.mfcc(sound.astype(float), sr=16000, n_mfcc=20)
            extracted_features_train.append(feature)

        for sound in segmented_data_test:
            extracted_features_test.append(librosa.feature.mfcc(sound.astype(float), sr=16000, n_mfcc=20))
    elif mode == 'mbe':

        for sound in segmented_data_train:
            feature = lmfe(sound.astype(float), 16000)
            extracted_features_train.append(feature)

        for sound in segmented_data_test:
            extracted_features_test.append(lmfe(sound.astype(float), 16000))
            
        for sound in segmented_data_unknown:
            extracted_features_unknown.append(lmfe(sound.astype(float), 16000))    
            
extract_feature('mbe')
x_train = np.array(extracted_features_train)
x_test = np.array(extracted_features_test)
x_unknown = np.array(extracted_features_unknown)


In [8]:
# temporary cell
print(x_train.shape)
y_train = np.array(raw_data_train[:,1])
print(y_train.shape)
print(x_test.shape)
y_test = np.array(raw_data_test[:,1])
print(y_test.shape)

print(x_unknown.shape)


(11307, 298, 40)
(11307,)
(473, 298, 40)
(473,)
(137, 298, 40)


In [5]:
raw_data_test[:,1].shape

(473,)

In [9]:
filename = 'mbe_3sec_random_default'
np.save('/home/alexandr/'+ filename, x_train)
np.save('/home/alexandr/y', y_train)
np.save('/home/alexandr/' + filename+ '_test', x_test)
np.save('/home/alexandr/y_test', y_test)
np.save('/home/alexandr/x_unknown',x_unknown)

In [7]:
y_test = np.array(raw_data_test[:,1])

np.save('/home/alexandr/y_test', y_test)

In [3]:
from keras.models import Sequential
from keras.layers import GRU, Dense, Activation, LSTM, Dropout
from keras.metrics import categorical_accuracy
from keras.utils import plot_model
import numpy as np
x_train = np.zeros((11307, 298, 40))

model = Sequential()

model.add(LSTM(256, input_shape = (x_train.shape[1:]), return_sequences = False))

model.add(Dense(8))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[categorical_accuracy])
plot_model(model, to_file=  '/home/alexandr/model.png', show_shapes=True)
